In [1]:
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import numpy as np
import deepcut
import codecs

# Read Input

In [2]:
# Read Input and sequence number
read_input = [i.replace("\n", '') for i in open("input.txt", encoding="utf8")]

input_sentence = [r.split("::")[1] for r in  read_input]
input_sentence_num = [r.split("::")[0] for r in  read_input]

print(input_sentence[:10])
print(input_sentence_num[:10])

['ยูนิคอร์นเป็นสัตว์ในตำนานซึ่งมีการอธิบายมาแต่โบราณว่าเป็นสัตว์ที่มีเขาเกลียวแหลมขนาดใหญ่ออกมาจากหน้าผาก มีการพรรณนายูนิคอร์นในตราโบราณของอารยธรรมลุ่มแม่น้ำสินธุและชาวกรีกโบราณกล่าวถึงยูนิคอร์นในบันทึกประวัติศาสตร์ธรรมชาติโดยนักเขียนหลายคน', 'เขาของสัตว์ตัวนั้นช่างสวยงาม', 'Horn แปลว่าเขา', 'Horn คือเขา', 'เขามีราคาแพง', 'เขาหาได้ยาก', 'Horny animal แปลว่าสัตว์มีเขา', 'ควายเป็นสัตว์มีเขา', 'กวางมีเขาสวยงาม', 'กวางมีเขาที่มีกิ่งสวยงาม']
['\ufeff1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


# Create word vector and Sentence vector

In [3]:
max_sentence_length = 21 # only allow 21 long of input

# find all available vocab
words = [[w for w in deepcut.tokenize(s) if w != ' '] for s in input_sentence]

In [4]:
# Extracted only the word between "เขา" 5 word
def split_from_kau(words):
    q = []
    flag_found_kau = False
    after_found = 0
    for w in words:
        q.append(w)
        if "เขา" in w:
            flag_found_kau = True
            after_found = len(q)
        elif flag_found_kau:
            if len(q) >= 21:
                break
        else:
            if len(q) > 10:
                q.pop(0)
    return q
# print(index_of_kau(words[0]))
words_split = [split_from_kau(w) for w in words]
print(words_split[-30:])
max_sentence_length = 21 # should not be larger than 21 word

[['ต้อง', 'รีบ', 'ข้าม', 'เขา', 'ก่อน', 'มืด'], ['เขา', 'ลูก', 'นี้', 'มี', 'สัตว์', 'ร้าย', 'อาศัย', 'อยู่'], ['ตั้งเต้น', 'บน', 'เขา', 'กัน', 'เถอะ'], ['ไม่', 'มี', 'ใคร', 'รอดกลับ', 'มา', 'ถ้า', 'ขึ้น', 'เขา', 'ลูก', 'นี้', 'ไป'], ['เจ้าแน่ใจ', 'ว่า', 'จะ', 'ตาม', 'ข้า', 'ขึ้น', 'ไป', 'บน', 'เขา'], ['ฉัน', 'เห็น', 'คน', 'ขับ', 'เครื่อง', 'ร่อน', 'ลอย', 'ข้าม', 'เขา', 'ไป'], ['เครื่อง', 'บิน', 'บิน', 'ข้าม', 'เขา', 'ไป', 'อย่าง', 'ยากลำบาก'], ['เขา', 'ใกล้', 'ถล่ม', 'ลง', 'มา', 'แล้ว', 'รีบ', 'หนี', 'เร็ว'], ['จังหวัด', 'ของ', 'เขา', 'ล้อม', 'รอบ', 'ไป', 'ด้วย', 'ทะเล'], ['เขา', 'นั่น', 'คง', 'ราคา', 'ดี', 'ไม่', 'น้อย', 'เลย'], ['เขา', 'หน่ะ', 'ช่าง', 'เป็น', 'คน', 'ที่', 'ดี'], ['ฉัน', 'จะ', 'ขึ้น', 'ไป', 'บน', 'ยอด', 'เขาเอเวอร์เรส'], ['เขา', 'ชิ้น', 'นั่น', 'คง', 'แพง', 'หน้า', 'ดู'], ['เขา', 'ต้อง', 'เป็น', 'ราชา', 'โจร', 'สลัด', 'ได้', 'แน่นอน'], ['เขา', 'ของ', 'วัว', 'ตัว', 'นั้น', 'ช่าง', 'งดงาม'], ['เขา', 'วาน', 'ให้', 'หนู', 'เป็น', 'สาย', 'ลับ'], ['ภูเขา', 'ที่', 'สูง', 'ท

In [5]:
vocab = set([w for s in words_split for w in s])
pretrained_word_vec_file = open('cc.th.300.vec', 'r',encoding = 'utf-8-sig')
count = 0
vocab_vec = {}
for line in pretrained_word_vec_file:
    if count > 0:
        line = line.split()
        if(line[0] in vocab): 
            vocab_vec[line[0]] = line[1:]
    count = count + 1

In [6]:
# fill the initial with 0
word_vector_length = 300
word_vectors = np.zeros((len(words_split),max_sentence_length,word_vector_length))
sample_count = 0
for s in words_split:
    word_count = 0
    for w in s:
        try:
            word_vectors[sample_count,max_sentence_length-word_count-1,:] = vocab_vec[w]
            word_count = word_count+1
        except:
            pass
    sample_count = sample_count+1

# Create Dictlabel prepare to convert it back

In [7]:
dict_label = {"H":0,"P":1,"M":2}
dict_label_reverse = {v:k for k,v in dict_label.items()}
print(dict_label)
print(dict_label_reverse)

{'H': 0, 'P': 1, 'M': 2}
{0: 'H', 1: 'P', 2: 'M'}


# Load model

In [14]:
#Load model
model = load_model('model.h5')

model.summary()

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 21, 300)]         0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 44)                56848     
_________________________________________________________________
dropout_3 (Dropout)          (None, 44)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 135       
Total params: 56,983
Trainable params: 56,983
Non-trainable params: 0
_________________________________________________________________


# Predict !!

In [15]:
y_pred = model.predict(word_vectors)
print(y_pred[:10])

[[1.0000000e+00 0.0000000e+00 1.5290514e-24]
 [9.9999964e-01 0.0000000e+00 3.0625014e-07]
 [1.0000000e+00 0.0000000e+00 8.5507566e-21]
 [1.0000000e+00 0.0000000e+00 1.0160175e-18]
 [9.9920505e-01 5.7336895e-22 7.9497776e-04]
 [1.0000000e+00 0.0000000e+00 8.0681456e-10]
 [1.0000000e+00 0.0000000e+00 6.0957825e-17]
 [1.0000000e+00 0.0000000e+00 6.1928508e-21]
 [1.0000000e+00 0.0000000e+00 9.6235730e-26]
 [1.0000000e+00 0.0000000e+00 7.8919094e-22]]


In [16]:
def find_index_of_max(l):
    i = (0,l[0])
    for y in enumerate(l):
        if i[1] < y[1]:
            i = y
    return i[0]

labels = [dict_label_reverse[find_index_of_max(pred)] for pred in y_pred]
print(labels)

['H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'P', 'H', 'H', 'H', 'H', 'H', 'H', 'P', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'M', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H',

# Write Results

In [17]:
with codecs.open("ans.txt", "w", "utf-8") as writer:
    for res in zip(input_sentence_num, labels):
        writer.write(res[0]+"::"+res[1]+"\n")